In [3]:
import sympy
import numpy as np
from dataclasses import dataclass
from typing import List
from IPython.display import SVG

In [180]:
from sympy.vector import Vector

In [183]:
from sympy.abc import a,b,c,d

In [184]:
A = (a,b)
B = (c,d)

In [187]:
Point = sympy.Function('Point')
Line = sympy.Function('Line')
Extrude = sympy.Function('Extrude')
Polygon = sympy.Function('Polygon')

In [190]:
import sympy.vector

In [192]:
from sympy import Matrix

In [194]:
Matrix([1,2])

Matrix([
[1],
[2]])

In [209]:
def make_rect(origin, offset, depth):
    a = Point(origin[0],origin[1])
    b = Point(origin[0]+offset[0],origin[0]+offset[1])
    l = Line(a, b)
    rect = Extrude(l, depth)
    return rect

In [210]:
x,y,z,a,b,c = sympy.symbols('x y z a b c')

In [211]:
rect = make_rect((x,y), (a,b), c)

In [212]:
sympy.sqrt(x)

sqrt(x)

Line(start=Point(x=x, y=y), end=Point(x=a + x, y=b + x))

In [12]:
from sympy import sqrt

@dataclass
class Number:
    val: float
    

num = Number

def box_union(*boxes):
    points = np.concatenate(boxes, 0)
    return np.stack([points.min(axis=0), points.max(axis=0)])

class Shape:
    pass

@dataclass
class Point(Shape):
    x: num
    y: num

    def eval(self):
        return f'<circle cx="{self.x}" cy="{self.y}" r="1" fill="green"/>'

    def box(self):
        return np.array([[self.x,self.y],[self.x,self.y]])

    def get_params(self):
        return [self.x, self.y]

@dataclass
class Line(Shape):
    start: Point
    end: Point

    def eval(self):
        return f'<line x1="{self.start.x}" y1="{self.start.y}" x2="{self.end.x}" y2="{self.end.y}" stroke-width="1" stroke="red"/>'
    
    def length(self):
        return sqrt((self.end.x - self.start.x)**2 + (self.end.y - self.start.y)**2)

    def box(self):
        return box_union(self.start.box(), self.end.box())

    def get_params(self):
        return self.start.get_params() + self.end.get_params()


@dataclass
class Extrude(Shape):
    line: Line
    distance: num

    def poly(self):
        norm = self.line.length()
        perp_x = self.distance * (self.line.start.x - self.line.end.x) / norm
        perp_y = self.distance * (self.line.end.x - self.line.start.x) / norm
        
        rect = Polygon([
           self.line.start,
           self.line.end,
           Point(self.line.end.x + perp_x, self.line.end.y + perp_y),
           Point(self.line.start.x + perp_x, self.line.start.y + perp_y)
        ])

        return rect

    def eval(self):
        return self.poly().eval()

    def box(self):
        return self.poly().box()

    def get_params(self):
        return 

@dataclass
class Polygon(Shape):
    points: List[Point]

    def eval(self):
        points_str = " ".join(f'{p.x},{p.y}' for p in self.points)
        return f'<polygon points="{points_str}" fill="blue" stroke="none"/>'

    def box(self):
        return box_union(*[p.box() for p in self.points])

a = Point(0,0)
b = Point(1,1)
l = Line(a,b)
rect = Extrude(l, 1)
rect.eval()

@dataclass
class Rectangle:
    c1: Point
    c2: Point

    def box(self):
        return box_union(self.c1.box(), self.c2.box())

    def eval(self):
        w = self.c2.x - self.c1.x
        h = self.c2.y - self.c1.y
        return f'<rect x="{self.c1.x}" y="{self.c1.y}" width="{w}" height="{h}" fill="blue"/>'


@dataclass
class Box:
    w: num
    h: num

    def box(self):
        return np.array([[0,0], [self.w, self.h]])
    
    def eval(self):
        return f'<rect width="{self.w}" height="{self.h}" fill="blue"/>'


@dataclass
class ShapeList(Shape):
    shapes: List[Shape]

    def box(self):
        return box_union(*[s.box() for s in self.shapes])
    
    def eval(self):
        contents_svg = '\n'.join([s.eval() for s in self.shapes])
        return f'<g>{contents_svg}</g>'

@dataclass
class Translate(Shape):
    shape: Shape
    x: num
    y: num

    def box(self):
        return self.shape.box() + np.array([self.x, self.y])

    def eval(self):
        return f'<g transform="translate({self.x} {self.y})">{self.shape.eval()}</g>'

@dataclass
class Scale(Shape):
    shape: Shape
    s: num

    def box(self):
        return self.shape.box()*self.s

    def eval(self):
        return f'<g transform="scale({self.s} {self.s})">{self.shape.eval()}</g>'

def draw(shape):
    shape_svg = shape.eval()
    box = shape.box()
    min_corner = box[0]
    box_size = box[1] - box[0]
    view_box = ' '.join([' '.join(str(x) for x in min_corner - np.array([1,1])), ' '.join(str(x) for x in box_size*2)])

    return f'<svg viewBox="{view_box}" xmlns="http://www.w3.org/2000/svg">{shape_svg}</svg>'

In [15]:
SVG(data=draw(Rectangle(Point(0,0), Point(5,5))))

In [177]:
a = Point(0,0)
b = Point(10, 10)
l = Line(a, b)
box = Extrude(l, -5)
l2 = Scale(Translate(l, 10, -4), .5)
shapes = ShapeList([box, l, l2, a ,b])
SVG(data=draw(shapes))

In [148]:
draw(l2)[90:96]

'2)><li'

In [150]:
draw(l2)

'<svg viewBox="-2 -2 10 10" xmlns="http://www.w3.org/2000/svg"><g transform="translate(-2 -2)><line x1="0" y1="0" x2="10" y2="10" stroke-width="1" stroke="red"/></g></svg>'

In [2]:
# numbers
# add
# line
# extrude

In [29]:
class Line:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
    def __call__(self):
        return f'line({self.x1()}, {self.y1()}, {self.x2()}, {self.x2()})'

class Number:
    def __init__(self, n):
        self.n = n
    def __call__(self):
        return self.n

from math import sqrt

class Polygon:
    def __init__(self, *points):
        self.points = points
    
    def __call__(self):
        points_str = " ".join(f'{str(a)},{str(b)}' for (a,b) in self.points)
        return f'<polygon points="{points_str}"/>'

class Extrude:
    def __init__(self, l, d):
        self.l = l
        self.d = d
    def __call__(self):
        x1 = self.l.x1()
        x2 = self.l.x2()
        y1 = self.l.y1()
        y2 = self.l.y2()

        perp_y = (y1 - y2)
        perp_x = (x2 - x1)
        perp_norm = sqrt(perp_y**2 + perp_x**2)
        n_perp_y = perp_y / perp_norm
        n_perp_x = perp_x / perp_norm

        return Polygon(
            (x1,y1),
            (x2,y2), 
            (x1+n_perp_x*self.d(), y1+n_perp_y*self.d()),
            (x2+n_perp_x*self.d(), y2+n_perp_y*self.d())
        )()

In [30]:
rect = Extrude(Line(Number(0),Number(0), Number(1), Number(2)), Number(1))

In [31]:
rect()

'<polygon points="0,0 1,2 0.4472135954999579,-0.8944271909999159 1.4472135954999579,1.1055728090000843"/>'

In [ ]:
# We want to go from a program text to an lambda plus a set of parameters. The lambda should generate an SVG using those parameters
